In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

visits_df=pd.read_excel('/kaggle/input/twcasestudy/TWCaseStudyDataInsightsAnalystSiteMerchUpdatedSept23.xlsx',sheet_name="Visits")
sales_df=pd.read_excel('/kaggle/input/twcasestudy/TWCaseStudyDataInsightsAnalystSiteMerchUpdatedSept23.xlsx',sheet_name="Sales")
avail_skus_df=pd.read_excel('/kaggle/input/twcasestudy/TWCaseStudyDataInsightsAnalystSiteMerchUpdatedSept23.xlsx',sheet_name="Available_SKUs")

In [ ]:
#To Identify the alignment, we need to identify the customer demands. The customer demands are
#mentioned in visits

#Check for each catagory the most number of visits and in which month.
from pandasql import sqldf
pysql = lambda q: sqldf(q, globals())

def run_and_return_query(query) -> pd.DataFrame:
    df=pysql(query)
    
    return df

In [ ]:
#what is customers demand
#how can we define it or identify it.

available_sku_query="""
        select
        vt.category,
        vt.PriceStaggered as CustomerLookingForPrice,
        sd.PriceStaggered as SoldPrice,
        vt.month,
        sum(vt.Visits) as TotalVisits,
        sum(vt.UniqueVisitors) as TotalUniqueVisitors,
        ast.CurrentListings as available_units,
        sum(sd.UnitsSold) as TotalUnitsSold,
        sum(sd.UniqueCustomers) as TotalUniqueCustomers,
        ast.priceStaggered as AvailableUnitPriceStaggered
        from visits_df vt
        left join avail_skus_df ast
        on vt.category = ast.category and vt.PriceStaggered = ast.PriceStaggered
        left join sales_df sd
        on vt.category = sd.category and vt.PriceStaggered = sd.PriceStaggered
        and vt.month = sd.month
        group by vt.category,vt.PriceStaggered,vt.month
        order by vt.month, vt.PriceStaggered
"""
demand_sku_sales=run_and_return_query(available_sku_query)
df_demand=demand_sku_sales.fillna(0)
df_demand

In [ ]:
#From above, we can define vistors to customers conversion ratio i.e what is the ratio of product
#to convert unique visitors to customers
conversion_query="""
    select
    category,
    round(sum(TotalUniqueCustomers)/sum(TotalUniqueVisitors),2) as VisitorsToCustomersRatio
    from df_demand
    group by category
    order by VisitorsToCustomersRatio desc
"""

conversion_df=run_and_return_query(conversion_query)
conversion_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define categories for the ratio
categories = ['Low', 'Medium', 'High', 'VeryHigh']

# Define the boundaries for the categories
cut_bins = [0, 0.2, 0.5, 1, 4]  # Adjust these boundaries as needed

# Categorize the ratios based on the defined boundaries
conversion_df['Ratio_Category'] = pd.cut(conversion_df['VisitorsToCustomersRatio'], bins=cut_bins, labels=categories)

# Count the number of data points in each category
category_counts = conversion_df['Ratio_Category'].value_counts()

# Create a bar chart to visualize the ratio categories
plt.figure(figsize=(8, 6))
category_counts.plot(kind='bar', color='skyblue', zorder=3 )
plt.title('Visitor-to-Customer Ratio Categories')
plt.xlabel('Categories')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.grid(zorder=0, linestyle='dashed')
plt.show()

From Above, The Customer to Conversion ratio is low for about 180 categories,this also shows the 
potential demands  in these category.

In [ ]:
sql_query="""
    select
    category,
    month,
    CustomerLookingForPrice,
    TotalUniqueVisitors,
    available_units,
    AvailableUnitPriceStaggered
    from
    df_demand
    group by category
    having TotalUniqueVisitors>0 and available_units>0 and TotalUnitsSold=0
    order by TotalUniqueVisitors desc,Month desc
"""

run_and_return_query(sql_query)

Above table show the data of Category which are available in skus with same staggeredprice that consumers were looking for but still it remain unsold.

In [ ]:
#Seasons of Sales
sql_query="""
select * from 
(
select 
    category,
    items_sold,
    month,
    dense_rank() over ( partition by month order by items_sold desc) as Rank
    from
(
    select
    category,
    sum(TotalUnitsSold) as items_sold,
    month
    from
    df_demand
    group by category,month
    having items_sold >0 
    order by month, items_sold desc
) x
) y
where rank <=5
"""

df_top_sales_category=run_and_return_query(sql_query)
df1=df_top_sales_category.reset_index(drop=True)

df1